# **Stress evaluation**

In [63]:
%matplotlib widget

In [64]:
from bmcs_shear.dic_crack import DICGrid, DICAlignedGrid, DICCOR, DICStateFields
import ibvpy.api as ib
import numpy as np
import matplotlib.pylab as plt
from matplotlib import cm
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [65]:
dic_grid = DICGrid(n_x=59, n_y=15, d_x=22, d_y=22,
                   start_t=0, end_t=1, U_factor=100, dir_name='B1_TV1', grid_column_first=False, 
                   grid_number_vertical=False)

In [66]:
dsf = DICStateFields(dic_grid=dic_grid)

In [67]:
dsf.tmodel.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.001)

In [68]:
dsf.hist.vis_record = {
  'strain': Vis3DTensorField(var='eps_ab'),
  'stress': Vis3DTensorField(var='sig_ab'),
#   'damage': Vis3DStateField(var='omega')
}

In [69]:
dsf.eval()

In [70]:
if False:
    from mayavi import mlab
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    scene.scene.background = (1.0, 1.0, 1.0)
    scene.scene.foreground = (0.0, 0.0, 0.0)
    scene.scene.z_plus_view()
    scene.scene.parallel_projection = True
#     strain_viz = Viz3DTensorField(vis3d=dsf.hist['strain'])
#     strain_viz.setup()
#     strain_viz.warp_vector.filter.scale_factor = 20
#     strain_viz.plot(0)
    stress_viz = Viz3DTensorField(vis3d=dsf.hist['stress'])
    stress_viz.setup()
    stress_viz.warp_vector.filter.scale_factor = 20
    stress_viz.plot(0)
#     damage_viz = Viz3DScalarField(vis3d=dsf.hist['damage'])
#     damage_viz.setup()
#     damage_viz.warp_vector.filter.scale_factor = 20
#     damage_viz.plot(0)
    mlab.show()

# Structured data of the damage field

In [71]:
t_idx = 25
kappa_Emr = dsf.hist.state_vars[t_idx][0]['kappa']
omega_Emr = dsf.hist.state_vars[t_idx][0]['omega']

In [72]:
phi_Emab = dsf.tmodel._get_phi_Emab(kappa_Emr)

In [73]:
omega_Em = np.max(omega_Emr, axis=-1)

In [74]:
X_pa = dsf.xmodel.x_Ema.reshape(-1,2)

In [75]:
fig, ax = plt.subplots(1,1)
ax.scatter(*X_pa.T, s=15*omega_Em.flatten(), color='red', alpha=0.4)
ax.axis('equal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Rearrange the regular element grid with integration points into a 2D point grid  

In [76]:
n_E, n_F = dic_grid.n_x-1, dic_grid.n_y-1
x_EFmna = dsf.xmodel.x_Ema[:,(0,3,1,2)].reshape(n_E, n_F, 2, 2, 2)
x_EmFna = np.einsum('EFmna->EmFna', x_EFmna)
x_MNa = x_EmFna.reshape(2*n_E, 2*n_F, 2)
x_aMN = np.einsum('MNa->aMN', x_MNa)

Rearrange the damage field

In [77]:
omega_EFmn = omega_Em[:,(0,3,1,2)].reshape(n_E, n_F, 2, 2)
omega_EmFn = np.einsum('EFmn->EmFn', omega_EFmn)
omega_MN = omega_EmFn.reshape(2*n_E, 2*n_F)

Rearrange the strain field

In [78]:
U_o = dsf.hist.U_t[t_idx]
eps_Emab = dsf.xmodel.map_U_to_field(U_o)
eps_EFmnab = eps_Emab[:, (0, 3, 1, 2)].reshape(n_E, n_F, 2, 2, 2, 2)
eps_EmFnab = np.einsum('EFmnab->EmFnab', eps_EFmnab)
eps_MNab = eps_EmFnab.reshape(2*n_E, 2*n_F, 2, 2)
delta_23 = np.array([[1, 0, 0],[0, 1, 0]])
eps_MNab = np.einsum('MNab,ac,bd->MNcd', eps_MNab, delta_23, delta_23)

In [79]:
eps_MNa, _ = np.linalg.eig(eps_MNab)
max_eps_MN = np.max(eps_MNa, axis=-1)
fig, ax = plt.subplots(1, 1, figsize=(8,3))
cs = ax.contour(x_aMN[0], x_aMN[1], max_eps_MN, cmap=cm.coolwarm)
ax.clabel(cs, inline=1, fontsize=10)
ax.axis('equal')
np.max(max_eps_MN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.020816914234722154

In [80]:
sig_Emab, _ = dsf.tmodel.get_corr_pred(eps_Emab, 1, kappa_Emr, omega_Emr)
sig_EFmnab = sig_Emab[:,(0,3,1,2)].reshape(n_E, n_F, 2, 2, 2, 2)
sig_EmFnab = np.einsum('EFmnab->EmFnab', sig_EFmnab)
sig_MNab = sig_EmFnab.reshape(2*n_E, 2*n_F, 2, 2)
delta_23 = np.array([[1, 0, 0],[0, 1, 0]])
sig_MNab = np.einsum('MNab,ac,bd->MNcd', sig_MNab, delta_23, delta_23)

In [81]:
sig_MNa, _ = np.linalg.eig(sig_MNab)
max_sig_MN = np.max(sig_MNa, axis=-1)
fig, ax = plt.subplots(1, 1, figsize=(8,3))
cs = ax.contour(x_aMN[0], x_aMN[1], max_sig_MN, cmap=cm.coolwarm)
ax.clabel(cs, inline=1, fontsize=10)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4.649147038914116, 1271.3508529610858, 4.649147038914116, 303.3508529610859)

In [82]:
from matplotlib import cm
fig, ax = plt.subplots(1, 1, figsize=(8,3))
ax.contour(x_aMN[0], x_aMN[1], omega_MN, cmap=cm.coolwarm)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4.649147038914116, 1271.3508529610858, 4.649147038914116, 303.3508529610859)

In [83]:
phi_EFmnab = phi_Emab[:, (0, 3, 1, 2)].reshape(n_E, n_F, 2, 2, 2, 2)
phi_EmFnab = np.einsum('EFmnab->EmFnab', phi_EFmnab)
omega_MNab = np.identity(2)[np.newaxis, np.newaxis, ...] - phi_EmFnab.reshape(2*n_E, 2*n_F, 2, 2)
delta_23 = np.array([[1, 0, 0],[0, 1, 0]])
omega_MNab = np.einsum('MNab,ac,bd->MNcd', omega_MNab, delta_23, delta_23)
omega_MNa, _ = np.linalg.eig(omega_MNab)
omega_MN = np.max(omega_MNa, axis=-1)
omega_MN[omega_MN < 0.3] = 0

In [84]:
fig, ax = plt.subplots(1, 1, figsize=(8,3))
ax.contour(x_aMN[0], x_aMN[1], omega_MN, cmap=cm.coolwarm)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4.649147038914116, 1271.3508529610858, 4.649147038914116, 303.3508529610859)

In [85]:
eps_Ema, _ = np.linalg.eig(eps_Emab)
max_eps_Em = np.max(eps_Ema, axis=-1)
arg_max_eps = np.argmax(max_eps_Em, axis=None)
arg_Em = np.unravel_index(arg_max_eps, max_eps_Em.shape)

In [86]:
arg_eps = eps_Emab[arg_Em]
arg_phi = phi_Emab[arg_Em]
arg_kappa = kappa_Emr[arg_Em]
arg_omega = omega_Emr[arg_Em]

In [87]:
dsf.tmodel.update_state_variables(arg_eps, arg_kappa, arg_omega)
dsf.tmodel._get_phi_Emab(arg_kappa)
beta_Emabcd = dsf.tmodel._get_beta_Emabcd(arg_phi)
D_Emijab = np.einsum(
            '...ijab, abef, ...cdef -> ...ijcd',
            beta_Emabcd, dsf.tmodel.D_abef, beta_Emabcd
        )
sig_Emab = np.einsum('...abef,...ef -> ...ab', D_Emijab, arg_eps)
sig_Emab

array([[-0.0161455 ,  0.12518003],
       [ 0.12518003, -0.44714317]])

In [88]:
arg_eps, arg_kappa, arg_phi

(array([[ 0.02012349,  0.00381175],
        [ 0.00381175, -0.00013636]]),
 array([0.02016638, 0.02051054, 0.0172427 , 0.01145562, 0.00516045,
        0.00055881, 0.        , 0.00174691, 0.0070861 , 0.01344815,
        0.01862211, 0.02080992, 0.01928962, 0.01455166, 0.00822967,
        0.00252067, 0.        , 0.        , 0.00415554, 0.01027817,
        0.01631435, 0.02016638]),
 array([[ 0.02172997, -0.05657826],
        [-0.05657826,  0.35255155]]))

In [89]:
kappa_zero = np.zeros_like(arg_kappa)
omega_zero = np.zeros_like(arg_omega)
eps_test = np.zeros_like(arg_eps)
eps_range = np.linspace(0,0.02,100)
sig_range = []
for eps_i in eps_range:
    eps_test[0,0] = eps_i
    arg_sig, _ = dsf.tmodel.get_corr_pred(eps_test, 1, kappa_zero, omega_zero)
    sig_range.append(arg_sig)
fig, ax = plt.subplots(1,1)
ax.plot(eps_range, np.array(sig_range)[:,0,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
from numpy import mgrid, empty, sin, pi
from tvtk.api import tvtk
from mayavi import mlab

In [91]:
# The actual points.
pts_shape = x_aMN[0].shape + (1,)
pts = np.empty(pts_shape + (3,), dtype=float)
pts[..., 0] = x_aMN[0, ..., np.newaxis]
pts[..., 1] = x_aMN[1, ..., np.newaxis]
pts[..., 2] = omega_MN[..., np.newaxis] * 100 #  np.zeros_like(pts[...,1])
pts = pts.transpose(2, 1, 0, 3).copy()
pts.shape = int(pts.size / 3), 3

In [92]:
if False:
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = np.einsum('MN...->NM...', omega_MN).ravel()
    sg.point_data.scalars.name = 'damage'
    sg.point_data.tensors = np.einsum('MN...->NM...', eps_MNab).reshape(-1,9)
    sg.point_data.tensors.name = 'strain'
    sg.point_data.tensors = np.einsum('MN...->NM...', sig_MNab).reshape(-1,9)
    sg.point_data.tensors.name = 'stress'
    sg.point_data.tensors = np.einsum('MN...->NM...', omega_MNab).reshape(-1,9)
    sg.point_data.tensors.name = 'omega'
    # Now visualize the data.
    d = mlab.pipeline.add_dataset(sg)
    iso = mlab.pipeline.iso_surface(d)
    surf = mlab.pipeline.surface(d)
    mlab.show()

# Interpolate damage field

In [93]:
from scipy.interpolate import interp2d

In [94]:
x_0 = x_aMN[0, :, 0]
x_1 = x_aMN[1, 0, :]
f_omega = interp2d(x_0, x_1, omega_MN.T, kind='cubic')

In [95]:
xnew = np.linspace(x_0[-1], x_0[0], 116)
ynew = np.linspace(x_1[0], x_1[-1], 28)
xx_NM, yy_NM = np.meshgrid(xnew, ynew)

In [96]:
f_omega_NM = f_omega(xnew, ynew)
f_omega_NM[f_omega_NM < 0] = 0

In [97]:
# The actual points.
pts_shape = xx_NM.shape + (1,)
pts = np.empty(pts_shape + (3,), dtype=float)
pts[..., 0] = xx_NM[...,np.newaxis]
pts[..., 1] = yy_NM[...,np.newaxis]
pts[..., 2] = f_omega_NM[..., np.newaxis] * 100 #  np.zeros_like(pts[...,1])
pts = pts.transpose(2, 1, 0, 3).copy()
pts.shape = int(pts.size / 3), 3
if True:
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = np.einsum('NM...->MN...', f_omega_NM).ravel()
    sg.point_data.scalars.name = 'damage'
    d = mlab.pipeline.add_dataset(sg)
    iso = mlab.pipeline.iso_surface(d)
    surf = mlab.pipeline.surface(d)
    mlab.show()

# Smoothing

In [98]:
def get_z_MN_ironed(x_JK, y_JK, z_JK, RR):
    n_J, n_K = x_JK.shape
    min_x, max_x = np.min(x_JK), np.max(x_JK)
    min_y, max_y = np.min(y_JK), np.max(y_JK)
    len_x = max_x - min_x
    len_y = max_y - min_y
    nx_JK = (x_JK - min_x) / len_x
    ny_JK = (y_JK - min_y) / len_y
    d_x = len_x / n_J
    d_y = len_y / n_K
    a_dxdy = d_x * d_y
    delta_x_JK = nx_JK[None, None, ...] - nx_JK[..., None, None]
    delta_y_JK = ny_JK[None, None, ...] - ny_JK[..., None, None]
    r2_n = (delta_x_JK**2 + delta_y_JK**2) / (2*RR**2) # (2*R**2)
    alpha_r_MNJK = np.exp(-r2_n)
    print(alpha_r_MNJK.shape, nx_JK[:,0].shape)
    np.trapz(alpha_r_MNJK, nx_JK[:,0], axis=-2)
    a_MN = np.trapz(np.trapz(alpha_r_MNJK, nx_JK[:, 0], axis=-2), ny_JK[0, :], axis=-1)
    normed_a_MNJK = np.einsum('MNJK,MN->MNJK', alpha_r_MNJK, 1 / a_MN)
    z_MNJK = np.einsum('...JK,JK->...JK',normed_a_MNJK, z_JK)
    z_MN = np.trapz(np.trapz(z_MNJK, nx_JK[:, 0], axis=-2), ny_JK[0,:], axis=-1)
    return z_MN

In [99]:
xx_MN, yy_MN, f_omega_MN = np.einsum('NM->MN', xx_NM), np.einsum('NM->MN', yy_NM), np.einsum('NM->MN', f_omega_NM)

In [100]:
fi_omega_MN = get_z_MN_ironed(xx_MN, yy_MN, f_omega_MN, 0.01)

(116, 28, 116, 28) (116,)


In [101]:
# The actual points.
pts_shape = xx_NM.shape + (1,)
pts = np.empty(pts_shape + (3,), dtype=float)
pts[..., 0] = xx_NM[...,np.newaxis]
pts[..., 1] = yy_NM[...,np.newaxis]
pts[..., 2] = fi_omega_MN.T[..., np.newaxis] * 100 #  np.zeros_like(pts[...,1])
pts = pts.transpose(2, 1, 0, 3).copy()
pts.shape = int(pts.size / 3), 3
if True:
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = fi_omega_MN.ravel()
    sg.point_data.scalars.name = 'damage'
    d = mlab.pipeline.add_dataset(sg)
    iso = mlab.pipeline.iso_surface(d)
    surf = mlab.pipeline.surface(d)
    mlab.show()

In [102]:
from scipy.signal import argrelextrema

# Crack tracing algorithm

In [111]:
N_range = np.arange(len(ynew))
omega_NM = fi_omega_MN.T
n_N, n_M = omega_NM.shape
omega_NM[omega_NM < 0.15] = 0  # cutoff small damage values
# smooth the landscape
# initial crack positions at the bottom of the zone
arg_C = argrelextrema(omega_NM[0, :], np.greater)[0]
# list of intervals decomposing the crack
intervals_Cp_ = []
# distance between right interval boundary and crack position
arg_C_shift_ = []
# list of crack horizontal indexes for each horizontal slice
arg_x_NC_ = [np.copy(arg_C)]
for N1 in N_range[1:]:
    # horizontal indexes of midpoints between cracks
    arg_C_left_ = np.hstack([[int(arg_C[0]/2)], np.array((0.25*arg_C[:-1] + 0.75*arg_C[1:]), dtype=np.int_)])
    arg_C_right_ = arg_C + 1
    # array of intervals - first index - crack, second index (left, right)
    intervals_Cp = np.vstack([arg_C_left_, arg_C_right_]).T
    # index distance from the right boundary of the crack interval
    arg_C_shift = np.array([
        np.argmax(omega_NM[N1, interval_p[-1]:interval_p[0]:-1])
        for interval_p in intervals_Cp
    ])
    # cracks, for which the next point could be identified
    C_shift = arg_C_shift>0
    # next index position of the crack
    arg_C[C_shift] = intervals_Cp[C_shift, -1] - arg_C_shift[C_shift]
    arg_x_NC_.append(np.copy(arg_C))
    # for debugging
    intervals_Cp_.append(intervals_Cp)
    arg_C_shift_.append(arg_C_shift)
arg_x_NC = np.array(arg_x_NC_)

In [112]:
n_C = arg_x_NC.shape[1]
arg_y_C = np.arange(n_N)
arg_y_NC = np.repeat(arg_y_C, n_C).reshape(n_N,-1)

In [113]:
x_MN, y_MN, omega_MN = xx_NM.T, yy_NM.T, omega_NM.T

In [114]:
fig, ax = plt.subplots(1,1, figsize=(8,3))
ax.plot(x_MN[arg_x_NC, arg_y_NC], y_MN[arg_x_NC, arg_y_NC], color='black', linewidth=3);
ax.contour(x_MN, y_MN, omega_MN, cmap=cm.coolwarm,
                antialiased=False)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4.649147038914116, 1271.3508529610858, 4.649147038914116, 303.3508529610859)

# Discussion of approaches to an automatic crack detection

**The below discussion is currently not relevant.**

The crack detection algorithm shown above
exploits the knowledge that cracks start from the bottom and propagate upwards and left.
Still I keep here for the case that a finite element approximation or variational approach
to crack detection might become relevant in the future.

The goal is to use the above field representation with the help of interpolation functions
that can automatically detect the cracks and teeth at the ultimate state.

Interpolate the above grid using the shape functions
$$
e_E = N_{i} e_{Ei}
$$

The derivative of the principle tensile strain field can then be constructed as
$$
 \nabla_\boldsymbol{x} e_{E} = \nabla_\boldsymbol{x} N_i e_{Ei}
$$

Using the index notation $\boldsymbol{x} = x_a$, we can rewrite this equation as
$$
e_{E,a} = N_{i,a} e_{Ei}
$$

It is possible to pick also the direction of the principal tensile strain and then to 
use the approximation  

# Model component: **DICStrainGrid**

The above prototyping code has been implemented within the `DICStrainGrid` model component
as part of the `DICInspector`

In [59]:
from bmcs_shear.dic_crack import DICStrainGrid

In [60]:
dic_eps = DICStrainGrid(dic_grid=dic_grid)

In [61]:
dic_eps.interact()